## Analyze A/B Test Results

You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).  **Please save regularly.**

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [118]:
import pandas as pd
import numpy as np
import random 
import matplotlib.pyplot as plt
%matplotlib inline 
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42) 

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [119]:
df = pd.read_csv('ab_data.csv') 

In [120]:
df.head()

user_id                   timestamp      group landing_page  converted
0   851104  2017-01-21 22:11:48.556739    control     old_page          0
1   804228  2017-01-12 08:01:45.159739    control     old_page          0
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4   864975  2017-01-21 01:52:26.210827    control     old_page          1

In [121]:
df.group.unique()

array(['control', 'treatment'], dtype=object)

In [122]:
df.landing_page.unique()

array(['old_page', 'new_page'], dtype=object)

In [123]:
df.converted.unique()

array([0, 1])

b. Use the cell below to find the number of rows in the dataset.

In [124]:
print(f'Rows: {df.shape[0]}')

Rows: 294478


c. The number of unique users in the dataset.

In [125]:
unique_users = df.user_id.nunique()
print(f'No of uique users: {unique_users}')

No of uique users: 290584


d. The proportion of users converted.

In [126]:
#since the columns contains just ones and zeros , we can just find the average of the converted column
prop_converted = df.converted.mean()
f'prop_converted: {prop_converted}'  

'prop_converted: 0.11965919355605512'

e. The number of times the `new_page` and `treatment` don't match.

In [127]:
treatment_mismatch = df.query('landing_page == "old_page" and group == "treatment"')
control_mismatch = df.query('landing_page == "new_page" and group == "control"')

In [128]:
total_mismatched = treatment_mismatch.shape[0] + control_mismatch.shape[0]
total_mismatched

3893

f. Do any of the rows have missing values?

In [129]:
df.isnull().any()

user_id         False
timestamp       False
group           False
landing_page    False
converted       False
dtype: bool

`2.` For the rows where **treatment** does not match with **new_page** or **control** does not match with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to figure out how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [130]:
df2=df.drop(treatment_mismatch.index)

In [131]:
df2= df2.drop(control_mismatch.index)

In [132]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

> All mismatched rows have been correctly removed

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [133]:
unique_users = df2.user_id.nunique()
unique_users

290584

b. There is one **user_id** repeated in **df2**.  What is it?

In [134]:
duplicated_user = df2[df2.user_id.duplicated() == True]
duplicated_user

user_id                   timestamp      group landing_page  converted
2893   773192  2017-01-14 02:55:59.590927  treatment     new_page          0

> user with id 2893 was  repeated

c. What is the row information for the repeat **user_id**? 

In [135]:
print('Info of Repeated User:\n')
f' {duplicated_user.to_dict()}'

Info of Repeated User:



" {'user_id': {2893: 773192}, 'timestamp': {2893: '2017-01-14 02:55:59.590927'}, 'group': {2893: 'treatment'}, 'landing_page': {2893: 'new_page'}, 'converted': {2893: 0}}"

> `2893` is the user id

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [136]:
df2.drop(2893, axis=0, inplace=True) 

`4.` Use **df2** in the cells below to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [137]:
conversion_prob = (df2.converted==1).mean()
conversion_prob

0.11959708724499628

In [138]:
f'Total Probabality of conversion is {conversion_prob} '

'Total Probabality of conversion is 0.11959708724499628 '

b. Given that an individual was in the `control` group, what is the probability they converted?

In [139]:
# let's filter the converted people in the control group
conversion_prob_given_control = df2.query('group =="control"') 
# let us find the mean of the conversion.. since our conversion column is 1 & 0's
conversion_prob_given_control = conversion_prob_given_control.converted.mean() 

In [140]:
f'Probability of Conversion Given Control: {conversion_prob_given_control}' 

'Probability of Conversion Given Control: 0.1203863045004612'

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [141]:
# let's filter the converted people in the treatment group
conversion_prob_given_treatment = df2.query('group == "treatment"')
# let us find the mean of the conversion.. since our conversion column is 1 & 0's
conversion_prob_given_treatment = conversion_prob_given_treatment.converted.mean()

In [142]:
f'Probability of Conversion Given Treatment: {conversion_prob_given_treatment}'

'Probability of Conversion Given Treatment: 0.11880806551510564'

d. What is the probability that an individual received the new page?

In [143]:
newpage_prob = (df['landing_page'] == 'new_page').mean()

In [144]:
f'Probability of an individual recieving the new page: {newpage_prob}' 

'Probability of an individual recieving the new page: 0.5'

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

> ### ANswer

**From the probability results calculated from the above, it is obvious that there is just a little difference in 
the users converted in each group. Also there is only 50% chance that new page leads to conversion.***

***We need to form an hypothesi,s perform sampling  to achieve normal distribution for a null hypothesis and test it's statistical significance.
Hence There is no concrete evidence to state that the new treatment page leads to more conversions until we carry out an hypotheis test**

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

**Put your answer here.**

> `Observation`

> **if the type 1 error rate is where the new page is better it means the new page being better than old page would be our alternative hypothesis**

> Hence we would have the following: 

$$h_o: p_{new} - p_{old}  <= 0 $$

$$h_1: p_{new} - p_{old} >0 $$ 
$$\alpha =0.05$$ 

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

In [145]:
df2_sample = df2.sample(df2.shape[0], replace=True)
df2_sample_mean = df2_sample.converted.mean() 

a. What is the **conversion rate** for $p_{new}$ under the null? 

In [146]:
p_new = df2_sample_mean
p_new 

0.11934586900861713

b. What is the **conversion rate** for $p_{old}$  under the null? <br><br>

In [147]:
p_old = df2_sample_mean
p_old

0.11934586900861713

c. What is $n_{new}$, the number of individuals in the treatment group?

In [148]:
n_new = df2.query('group == "treatment"').user_id.nunique()
n_new

145310

d. What is $n_{old}$, the number of individuals in the control group?

In [149]:
n_old = df2.query('group == "control"').user_id.nunique()
n_old

145274

e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [150]:
def simulate_new():
    new_page_converted = np.random.choice([0,1], p=[1-p_new , p_new], size= n_new)
    return new_page_converted

f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [151]:
def simulate_old():
    old_page_converted = np.random.choice([0,1], p=[1-p_old , p_old], size=n_old,)
    return old_page_converted

g. Find $p_{new} - p_{old}$ for your simulated values from part (e) and (f).

In [152]:
p_new = simulate_new().mean()
p_new

0.11966141352969513

In [153]:
p_old = simulate_old().mean()
p_old

0.11925051970758704

In [154]:
diff_simul_vals = p_new -p_old

In [155]:
f'Difference ofsimulated values: {diff_simul_vals}'

'Difference ofsimulated values: 0.00041089382210808545'

> ### Observation

##### From the result of the simulated  difference in new page and old page conversion rate, 
##### One mmight begin to wonder whether our null hypothesis ``(where old page is better than new page)`` is  statistically significant
##### But let's perform  bootstrapping on our dataset (for normal distribution) to help us  decide whether or not to reject or fail to reject the null hypothesis

h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called **p_diffs**

In [ ]:
p_diffs = []
for _ in range(10000):
    p_diffs.append(simulate_new().mean() - simulate_old().mean())
p_diffs = np.array(p_diffs) 

In [ ]:
# let's take sneak peek at our values
p_diffs

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
plt.hist(p_diffs)
plt.axvline(diff_simul_vals, color='r')
plt.show()

> **From the above we can see that this normal distribution appears to follow our null hyothesis
> The sampled difference (negative) falls far below the zero tick in the x-axis**

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
df2.head()

In [ ]:
old_page_conversion = df2.query('landing_page == "old_page"').converted.mean()
new_page_conversion = df2.query('landing_page == "new_page"').converted.mean()

In [ ]:
atual_diff = new_page_conversion - old_page_conversion

In [ ]:
(p_diffs > atual_diff).mean()  

k. Please explain using the vocabulary you've learned in this course what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

##### Answer:
> ##### We justcalculated the p-value. 

> ##### The calculated p-value is above the alpha value of 5%. 

> ##### Since the simulation was from the null hypothesis 𝑝𝑛𝑒𝑤  - 𝑝𝑜𝑙𝑑 <= 0, the null hypothesis is statistically significant
> ##### The Old Page is more likely to result in increased Conversion over the new page

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
df2.head(2)

In [ ]:
import statsmodels.api as sm

converted_old = df2.query('landing_page == "old_page" and converted ==1').user_id.count()
converted_new = df2.query('landing_page == "new_page" and converted ==1').user_id.count()
n_old =  df2.query('landing_page == "old_page"').user_id.count()
n_new = df2.query('landing_page == "new_page"').user_id.count()

In [ ]:
converted_old

In [ ]:
converted_new

In [ ]:
n_old

In [ ]:
n_new

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](https://docs.w3cub.com/statsmodels/generated/statsmodels.stats.proportion.proportions_ztest/) is a helpful link on using the built in.

In [ ]:
from statsmodels.stats.proportion import proportions_ztest as pz_test

In [ ]:
counts = np.array([converted_old, converted_new])
nobs = np.array([n_old,n_new]) 
z_score, p_value =  pz_test(counts, nobs, alternative='smaller')

In [ ]:
print(z_score, p_value)

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

> ### Answer

**The z-score gives us the value of the standard deviation from the null distribution, while the p-value tells us the probability of getting this value or more extreme if the null is true which apparently is high here .**

**In this case, the z-score means we are 1.31 standard deviations from the mean. Since the p-value is 0.905 and is greater than our alpha of 0.05,**

***we can conclude that we should fail to reject the null hypothesis: the new landing page does not get better conversion rates than the old.**

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**Answer**

**Logistic Regression**

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df2.head()

In [ ]:
#let's copy our datasets into another for our regression model
df_regress = df2.copy()
(df_regress == df2).all()

In [ ]:
df_regress=df_regress.join(pd.get_dummies(df2['landing_page']))

In [ ]:
df_regress.head(1)

In [ ]:
df_regress.drop('old_page', axis=1, inplace=True)

In [ ]:
# add an ab_page column, which is 1 when an individual receives the treatment and 0 if control.
# the new page would be renamed as ab_page
df_regress.rename(columns={'new_page':'ab_page'}, inplace=True)

In [ ]:
df_regress.head()

In [ ]:
df_regress['intercept'] =1

c. Use **statsmodels** to instantiate your regression model on the two columns you created in part b., then fit the model using the two columns you created in part **b.** to predict whether or not an individual converts. 

In [ ]:
response = df_regress['converted'] 
predictors = df_regress[['intercept', 'ab_page']]

conv_model = sm.Logit(response, predictors)
conv_model_result = conv_model.fit()

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
conv_model_result.summary2()

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in **Part II**?

> ### Answer

***ab_page p-value = 0.1899.
In Part II, the p-value was 0.995***

***The reason for the difference in p-values is that Part II tested whether the probability of conversion was different on the new page vs. the old page. I.e. The conversion rate is the same or better for the old page than the new page.***

***However this p-value is looking at whether ab_page serves as a good predictor to determine conversion. that is, either of the pages (old vs. new) is a good predictor of conversion.***

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

> ### Answer

**Considering other factors to add to our regression model can help us imporove our prediction, and help us identify factors responsible for customer's conversion**


**The disadvantage is  associated with multicolinearity, whereby independent varibles are hase some sort of correlated relationship which can affect the performance of our prediction model**

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. You will need to read in the **countries.csv** dataset and merge together your datasets on the appropriate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
countries = pd.read_csv('countries.csv')

In [ ]:
countries.head(2)

In [ ]:
merged=df_regress.merge(countries, on='user_id', how='inner')

In [ ]:
merged.head()

In [ ]:
merged.country.unique()

In [ ]:
merged = merged.join(pd.get_dummies(merged.country))

In [ ]:
merged.head(1)

In [ ]:
#drop country column since we already have dummy values
merged.drop('country', axis=1, inplace=True)

In [ ]:
merged.head()

**_We would take country column:'CA' as our baseline for country_**

In [ ]:
merged['intercept'] =1
response = merged['converted']
predictors = merged[['intercept','ab_page','UK','US']]

In [ ]:
new_model = sm.Logit(response, predictors)
new_model_result = new_model.fit()

In [ ]:
new_model_result.summary2()

> ### Observation

**From the output of the model, we can deduce that country and page predictors are not statistically significant predictors of customers conversion given the new_page**

**Based on this relaization, we fail to reject the null hypothesis that the old page is better than the new page**

### Further actions

**I would be creating interaction to between page and country to further observe if these independent variables  would offer statistical significance ias predictors of  conversion given the nes page**

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
# create interactions for each predictor country and page
merged['ab_US'] = merged.ab_page * merged.US
merged['ab_UK'] = merged.ab_page * merged.UK

In [ ]:
res2=merged['converted']
pred2 = merged[['intercept','ab_US','ab_UK', 'ab_page', 'US', 'UK']]

In [ ]:
new_model2 = sm.Logit(res2, pred2)
new_model2_result = new_model2.fit()

In [ ]:
new_model2_result.summary2()

In [ ]:
res3 = merged['converted']
pred3 = merged[['intercept', 'ab_US','ab_UK']]

In [ ]:
model3 = sm.Logit(res3, pred3)
model3_result = model3.fit()

In [ ]:
model3_result.summary2()

> ### Conclusion

***Based on the various interaction model outputs above, we can observe that there are no right predictors for customer conversion given the new page**


**There is not enough statistical signifacnace from our preditors to predict if a customer converts or not given the new page**

**Our conclusion would therefore be that the old page is better than the new page**

<a id='conclusions'></a>
## Finishing Up

> Congratulations!  You have reached the end of the A/B Test Results project!  You should be very proud of all you have accomplished!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.


## Directions to Submit

> Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])